In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [101]:
#input term
trfname=input("Write your train data file name : ")
tstfname=input("Write your test data file name : ")
sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
if sep=='1':
    form=' '
elif sep=='2':
    form=','


hd=input("Select the data header format(1 = with header or 2 = no header): ")
if hd=='1':
    header='infer'
elif hd=='2':
    header=None


cl=input("Select column# where your y vlaues exist: ")
yno = int(cl)-1


train = pd.read_csv(trfname,sep=form,header=header)
test = pd.read_csv(tstfname,sep=form,header=header)

yCol=train.columns[yno]

Write your train data file name : pid.dat
Write your test data file name : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Select the data header format(1 = with header or 2 = no header): 2
Select column# where your y vlaues exist: 8


In [102]:
X = train.drop(yCol,1)
Y = train.iloc[:, yno]
df = pd.concat([X, Y], axis=1)

tX = test.drop(yCol,1)
tY = test.iloc[:, yno]
tdf = pd.concat([tX, tY], axis=1)
unqCnt = []
for i in range(X.shape[1]):
    unqCnt += [len(set(X.iloc[:,i]))]
uqCnt = np.array(unqCnt)

In [103]:
unqCnt = []
for i in range(X.shape[1]):
    unqCnt += [len(set(X.iloc[:,i]))]
uqCnt = np.array(unqCnt)

cateVarInd = np.where(uqCnt<=10)[0] # index of the categoral values
numerVarInd = np.where(uqCnt>10)[0] # index of the numerical values

In [104]:
def speLst(lst):
    n = len(lst)
    ctgry=[]
    for i in range(1,2**(n-1)):
        ctgry.append(list('{:#b}'.format(i)[2:].zfill(n)))

    boolCtgry = [[bool(int(y)) for y in x] for x in ctgry]

    g1 = []
    for i in boolCtgry:
        g1.append(sorted(np.array(lst)[i]))
    return g1

In [105]:
dicInd = dict()
for i in cateVarInd:
    dicInd.update({i:speLst(list(set(X.iloc[:,i])))})

In [106]:
def calcGini(vec):
    return 1 - (sum(vec==min(Y))/len(vec))**2-(sum(vec==max(Y))/len(vec))

In [107]:
def searchCoffcat(df,i):
    out = []
    for j in dicInd[i]:
        g1ind = j
        g2ind = [x for x in list(set(df.iloc[:,i])) if x not in g1ind]
        g1 = df[list(map(lambda x:x in g1ind,df.iloc[:,i]))]
        g2 = df[list(map(lambda x:x in g2ind,df.iloc[:,i]))]
        wGini = len(g1)*calcGini(g1[yCol])+len(g2)*calcGini(g2[yCol])
        out.append([wGini,j])  
    return sorted(out , key = lambda x : x[0])[0]

In [108]:
def searchCoff(df , cInd):
    out = []
    for i in sorted(list(set(df[cInd])))[:-1]:
        g1 = df[df[cInd] <= i]
        g2 = df[df[cInd] > i]
        wGini = len(g1)*calcGini(g1[yCol])+len(g2)*calcGini(g2[yCol])
        out.append([wGini,i])
    return sorted(out , key = lambda x : x[0])[0]

In [109]:
ret = []
for i in numerVarInd:
    ret.append([i]+searchCoff(df,i))

In [110]:
retCat = []
for i in cateVarInd:
    retCat.append([i]+searchCoffcat(df,i))

In [111]:
retall = ret + retCat

In [112]:
resCat = sorted(retall, key = lambda x : x[1])[0]

In [113]:
resCat

[1, 39.420875420875426, 146]

In [40]:
if resCat[0] in cateVarInd:
    g1 = df[list(map(lambda x:x in resCat[2], df.iloc[:,resCat[0]]))]
    g2 = df[list(map(lambda x:x not in resCat[2], df.iloc[:,resCat[0]]))]
    #g2 = df[df[res[0]] > res[2]]
else:
    g1 = df[df[resCat[0]] <= resCat[2]]
    g2 = df[df[resCat[0]] > resCat[2]]

In [41]:
if Counter(g1[yCol])[min(Y)] > Counter(g1[yCol])[max(Y)]:
    g1val = min(Y)
    g2val = max(Y)
else:
    g1val = max(Y)
    g2val = min(Y)

In [42]:
def getYhat(vec):
    if resCat[0] in cateVarInd:
        if vec[resCat[0]] in resCat[2]:
            return g1val
        else:
            return g2val
    else:
        if vec[resCat[0]] <= resCat[2]:
            return g1val
        else:
            return g2val

In [43]:
rslt= []
for i in range(X.shape[0]):
    rslt.append(getYhat(X.iloc[i,:]))
rslt = np.array(rslt)

cont = pd.crosstab(Y,rslt)
cont.columns.name = 'Resub Class'
cont.index.name = 'Actual Class'
acc = sum(np.diag(cont))/len(Y)
spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
predy = pd.DataFrame([range(1,(len(X)+1)),Y,rslt]).transpose()
predy.columns=['ID','Actual class','Resub pred']

In [23]:
trslt= []
for i in range(tX.shape[0]):
    trslt.append(getYhat(tX.iloc[i,:]))
trslt = np.array(trslt)

tcont = pd.crosstab(tY,trslt)
tcont.columns.name = 'Test Class'
tcont.index.name = 'Actual Class'
tacc = sum(np.diag(tcont))/len(tY)
tspe = tcont.iloc[0,0]/sum(tcont.iloc[0,:])
tsens = tcont.iloc[1,1]/sum(tcont.iloc[1,:])
tpredy = pd.DataFrame([range(1,(len(tX)+1)),tY,trslt]).transpose()
tpredy.columns=['ID','Actual class','Test pred']

In [72]:
resCat

[2, 381.4113401428271, ['Male']]

In [ ]:
fname = input('Write your output file name : ')

print(predy.to_string(index=False))
print(cont)
print("Overall accuracy = %0.3f" %(acc))
print("Sensitivity = %0.3f" %(sens))
print("Specificity = %0.3f" %(spe))

print(tpredy.to_string(index=False))
print(tcont)
print("Overall accuracy = %0.3f" %(tacc))
print("Sensitivity = %0.3f" %(tsens))
print("Specificity = %0.3f" %(tspe))

#file output
out = open(fname, 'w')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(predy.to_string(index=False),file=out)
    print("",file=out)
    print("Confusion Matrix (Resubstitution)",file=out)
    print(cont,file=out)
    print("",file=out)
    print("Model Summary (Resubstitution)",file=out)
    print("Overall accuracy = %0.3f" %(acc),file=out)
    print("Sensitivity = %0.3f" %(sens),file=out)
    print("Specificity = %0.3f" %(spe),file=out)
    print("",file=out)
    print(tpredy.to_string(index=False),file=out)
    print("",file=out)
    print("Confusion Matrix (Test)",file=out)
    print(tcont,file=out)
    print("",file=out)
    print("Model Summary (Test)",file=out)
    print("Overall accuracy = %0.3f" %(tacc),file=out)
    print("Sensitivity = %0.3f" %(tsens),file=out)
    print("Specificity = %0.3f" %(tspe),file=out)
out.close()

In [49]:
resCat[2]
a= ['1','2']

In [71]:
print('Tree Structure')
print('\tNode1 : %s in %s (%d,%d)' %(df.columns[[resCat[0]]][0],set(resCat[2]),Counter(train[yCol])[g1val],Counter(train[yCol])[g2val]))
print('\t Node2 : %s (%d,%d)' %(g1val,Counter(g1[yCol])[g1val],Counter(g1[yCol])[g2val]))
print('\t Node2 : %s (%d,%d)' %(g2val,Counter(g2[yCol])[g1val],Counter(g2[yCol])[g2val]))

Tree Structure
	Node1 : Sex in {'Male'} (1490,711)
	 Node2 : No (1364,367)
	 Node2 : Yes (126,344)


In [80]:
g1.iloc[:,resCat[0]][0]==resCat[2][0]

True

In [76]:
resCat

[2, 381.4113401428271, ['Male']]

In [82]:
print('\t Node2 : %s (%d,%d)' %(str(g1.iloc[:,resCat[0]][0]==resCat[2][0]),Counter(g1[yCol])[g1val],Counter(g1[yCol])[g2val]))
print('\t Node2 : %s (%d,%d)' %(str(g2.iloc[:,resCat[0]][0]==resCat[2][0]),Counter(g2[yCol])[g1val],Counter(g2[yCol])[g2val]))

	 Node2 : True (1364,367)


KeyError: 0

In [100]:
g2.iloc[:,resCat[0]].iloc[0]

'Female'